# Computing for Economists: Data Wrangling





In this module, we will have you work through the following datasets to analyze social connectedness in Europe. This module is inspired by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).

In [2]:
### Import Modules
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## Tidying the Data

1. The `data` folder contains data from the following sources. Read each of the data files into Python as Pandas DataFrames.
  - https://ec.europa.eu/eurostat/en/web/products-datasets/-/EDAT_LFSE_04 
    - This data contains the percent of population by educational attainment level, sex, and NUTS 2 regions in the European Union.
  - https://ec.europa.eu/eurostat/web/products-datasets/-/nama_10r_2hhinc
    - The data contains the income of households by NUTS 2 region.  
  - https://ec.europa.eu/eurostat/web/products-datasets/product?code=demo_r_pjanaggr3
    - Population by age, group, sex, and NUTS 2 region.
  - https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts
    - NUTS region shapefiles (use the geopandas library to read it as a GeoDataFrame).
  - https://media.githubusercontent.com/media/social-connectedness-index/euro_sci/master/_intermediate_data/geo_distance_dat.csv
    - Distance between pairs of NUTS 2 regions constructed by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).
  - `gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv` from https://data.humdata.org/dataset/social-connectedness-index
    - This file contains the social connectedness index between NUTS 2 regions and all Global Administrative Areas (GADM). We will only consider NUTS 2 to NUTS 2 connections. Social connectedness is a snapshot of Facebook links as of July 2019.
    - Note: this file is very large so we recommend reading the file directly into the notebook at the following link: https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv


In [137]:
path_education = "data/estat_edat_lfse_04.tsv"
path_income = "data/estat_nama_10r_2hhinc.tsv"
path_population = "data/estat_demo_r_pjanaggr3.tsv"
path_nuts_shapefile = "https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv"
path_geo_distance = "data/geo_distance_dat.csv"
path_sci = "data/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv"

df_education = pd.read_csv(path_education, sep='\t')
inc_data = pd.read_csv(path_income, sep='\t')
pop_data = pd.read_csv(path_population, sep='\t')
#df_shapefile = pd.read_csv(path_nuts_shapefile, sep='\t')
df_distance_nuts2 = pd.read_csv(path_sci, sep='\t')



2. Clean the education data (`estat_edat_lfse_04.tsv`) and put it into "tidy" format. Check that there is a unique non-missing key for each row. The resulting data should have five columns and 299,088 rows.

In [138]:
df_education[['freq', 'sex', 'isced11', 'age', 'unit', 'geo']] = df_education['freq,sex,isced11,age,unit,geo\\TIME_PERIOD'].str.split(',', expand=True)
df_education.drop(columns=['freq,sex,isced11,age,unit,geo\\TIME_PERIOD'], inplace=True)

df_education_clean = pd.melt(df_education, id_vars=['freq', 'sex', 'isced11', 'age', 'unit', 'geo'], var_name='year', value_name='Value')
df_education_clean['age'].replace('Y', '', regex=True, inplace=True)
df_education_clean['Value'] = pd.to_numeric(df_education_clean['Value'].str.extract(r'(\d+\.\d+|\d+)')[0], errors='coerce')
df_education_clean['year'] = df_education_clean['year'].astype(int)
df_education_clean.drop_duplicates(inplace=True)

df_education_clean = df_education_clean[df_education_clean['sex']=='T']
df_education_clean = df_education_clean[['isced11', 'age', 'geo', 'year', 'Value']]
df_education_clean


/var/folders/y8/xj185dtn4kgd924j3qb2gv240000gn/T/ipykernel_94240/1547174369.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_education_clean['age'].replace('Y', '', regex=True, inplace=True)


,isced11,age,geo,year,Value
24924,ED0-2,20-24,AT,2000,14.9
24925,ED0-2,20-24,AT1,2000,15.3
24926,ED0-2,20-24,AT11,2000,NaN
24927,ED0-2,20-24,AT12,2000,14.8
24928,ED0-2,20-24,AT13,2000,15.8
...,...,...,...,...,...
897259,ED5-8,30-34,UKM7,2023,NaN
897260,ED5-8,30-34,UKM8,2023,NaN
897261,ED5-8,30-34,UKM9,2023,NaN
897262,ED5-8,30-34,UKN,2023,NaN


3. Do the same for the income data (`estat_nama_10r_2hhinc.tsv`) and the population data (`estat_demo_r_pjanaggr3.tsv`).

In [139]:

inc_data[['freq', 'unit', 'direct', 'na_item', 'geo']] = inc_data['freq,unit,direct,na_item,geo\TIME_PERIOD'].str.split(',', expand=True)

inc_data = (
    inc_data
    .drop(columns=["freq", "freq,unit,direct,na_item,geo\TIME_PERIOD"])
    .melt(id_vars=['unit', 'direct', 'na_item', 'geo'], value_name="value", var_name="year")
    .assign(
        value = lambda x: pd.to_numeric(x["value"], errors="coerce"),
        year = lambda x: x["year"].astype(int)
    )
)

inc_data


pop_data[['freq', 'unit', 'sex', 'age', 'geo']] = pop_data['freq,unit,sex,age,geo\TIME_PERIOD'].str.split(',', expand=True)

pop_data = (
    pop_data
    .drop(columns=["freq", "freq,unit,sex,age,geo\TIME_PERIOD"])
    .melt(id_vars=['unit', 'sex', 'age', 'geo'], value_name="pop", var_name="year")
    .assign(
        pop = lambda x: pd.to_numeric(x["pop"], errors="coerce"),
        year = lambda x: x["year"].astype(int)
    )
)

pop_data


<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\T'
<>:16: SyntaxWarning: invalid escape sequence '\T'
<>:20: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:5: SyntaxWarning: invalid escape sequence '\T'
<>:16: SyntaxWarning: invalid escape sequence '\T'
<>:20: SyntaxWarning: invalid escape sequence '\T'
/var/folders/y8/xj185dtn4kgd924j3qb2gv240000gn/T/ipykernel_94240/3670749831.py:1: SyntaxWarning: invalid escape sequence '\T'
  inc_data[['freq', 'unit', 'direct', 'na_item', 'geo']] = inc_data['freq,unit,direct,na_item,geo\TIME_PERIOD'].str.split(',', expand=True)
/var/folders/y8/xj185dtn4kgd924j3qb2gv240000gn/T/ipykernel_94240/3670749831.py:5: SyntaxWarning: invalid escape sequence '\T'
  .drop(columns=["freq", "freq,unit,direct,na_item,geo\TIME_PERIOD"])
/var/folders/y8/xj185dtn4kgd924j3qb2gv240000gn/T/ipykernel_94240/3670749831.py:16: SyntaxWarning: invalid escape sequence '\T'
  pop_d

,unit,sex,age,geo,year,pop
0,NR,F,TOTAL,AL,1990,NaN
1,NR,F,TOTAL,AL0,1990,NaN
2,NR,F,TOTAL,AL01,1990,NaN
3,NR,F,TOTAL,AL011,1990,NaN
4,NR,F,TOTAL,AL012,1990,NaN
...,...,...,...,...,...,...
1061815,NR,T,Y_LT15,UKN12,2023,NaN
1061816,NR,T,Y_LT15,UKN13,2023,NaN
1061817,NR,T,Y_LT15,UKN14,2023,NaN
1061818,NR,T,Y_LT15,UKN15,2023,NaN


4. Outer join the percent of the population age 25 to 64 in a NUTS 2 region receiving a tertiary education with the income per capita in purchasing power standard (PPS). Then outer join the education and income data with population data for the total population (for sex and age). Only include data at the NUTS 2 level.

  To obtain the income per capita in PPS, the national account should be the "balance of primary incomes/national income, net" and the direction of flow should be the "balance".

In [142]:
df_education_tert_2564 = df_education_clean[df_education_clean['age'] == "25-64"] 
df_education_tert_2564 = df_education_tert_2564[df_education_tert_2564['isced11'] == "ED5-8"]

inc_data_pps = inc_data[inc_data['unit'] == "PPS_EU27_2020_HAB"]
join_1 = pd.merge(df_education_tert_2564, inc_data_pps, on=['geo', 'year'], how='outer')
join_1


,isced11,age,geo,year,Value,unit,direct,na_item,value
0,NaN,NaN,AT,1995,NaN,PPS_EU27_2020_HAB,BAL,B5N,NaN
1,NaN,NaN,AT,1995,NaN,PPS_EU27_2020_HAB,BAL,B6N,NaN
2,NaN,NaN,AT,1996,NaN,PPS_EU27_2020_HAB,BAL,B5N,NaN
3,NaN,NaN,AT,1996,NaN,PPS_EU27_2020_HAB,BAL,B6N,NaN
4,NaN,NaN,AT,1997,NaN,PPS_EU27_2020_HAB,BAL,B5N,NaN
...,...,...,...,...,...,...,...,...,...
26237,ED5-8,25-64,UKN0,2019,40.6,NaN,NaN,NaN,NaN
26238,ED5-8,25-64,UKN0,2020,NaN,NaN,NaN,NaN,NaN
26239,ED5-8,25-64,UKN0,2021,NaN,NaN,NaN,NaN,NaN
26240,ED5-8,25-64,UKN0,2022,NaN,NaN,NaN,NaN,NaN


5. For each pair of NUTS 2 regions, obtain the absolute difference in the two variables constructed in problem 4 in 2019. Drop the French islands (starts with NUTS 2 region code "FRY").


6. Now we will merge the the 2019 data with the SCI data and the pairwise distance data. Create a dataframe with one row for each pair of NUTS 2 regions (as two separate columns) and the following variables:

*   Log scaled SCI
*   Log distance (add 1 to avoid zeros)
*   Percent tertiary education for User and Friend NUTS 2
*   Income per capita for User and Friend NUTS 2
*   Population for User and Friend NUTS 2
*   Absolute difference in tertiary education between User and Friend NUTS 2
*   Absolute difference in income per capita between User and Friend NUTS 2

  Note that the SCI data contains GADM areas, please only keep the NUTS 2 to NUTS 2 rows. Keeps all rows in the SCI data.


## Descriptive Statistics


1.   Each country is represented by the first two letters of the NUTS 2 index. Present a country-level population-weighted summary table for the percent people with tertiary education and income per capita in 2019. Include the mean, standard deviation, min, and max. Round all numeric outputs to two decimal places.

2. The SCI is scaled from 1 to 1,000,000 and measures the relative probability of a Facebook friendship link ([source](https://data.humdata.org/dataset/social-connectedness-index?)). If a number is two times higher, then a friendship link between two users from those NUTS 2 regions is twice as likely. Calculate  the mean, std, min, and max of log(scaled SCI) by country and merge it with the summary table from the previous part.

3. Compute the correlation between log(scaled SCI), log(distance), user region education and income, as well as differences in education and income between NUTS 2 regions. Is it higher or lower than what you expected?

## Data Visualization

1. We will ask you to plot two maps of social connections of (1) a country and (2) a NUTS 2 region to all other NUTS 2 regions. Remember the French islands should be dropped (these islands are far away from continental Europe and will therefore make the maps harder to visualize).

  * For the chosen country, aggregate the SCI to produce country to NUTS 2 region measures. Scaled SCI can be aggregated correctly using population shares following [SCI documentation](https://data.humdata.org/dataset/social-connectedness-index). For countries $i, j$ with regions $\{r_i\}, \{r_j\}$, aggregate SCI is given by:

  $$SCI_{i,j} = \sum_{\{r_i\}}\sum_{\{r_j\}} PopShare_{r_i} \times PopShare_{r_j} \times SCI_{r_i, r_j}$$

  where $PopShare_{r_i}$ is the share of country $i$'s population in region $r_i$.

  * Merge the aggregated SCI data with the GeoDataFrame (keep only NUTS 2 regions) for the applicable geographies.
  * Construct the bins for the map legend by first selecting a baseline SCI cutoff (e.g. 20th percentile of the **total data** that you are working with). Then calculate the subsequent cutoffs as $2\times$, $3\times$, $5\times$, $10\times$, $25\times$, $100\times, 1000\times$ that of the baseline cutoff. Feel free to use another legend system as long as there is justification.
  * Map directly using the GeoPandas library, or feel free to explore interactive maps in Plotly.

  Recall the principles of software engineering (consider writing a function for code that may be reused).

2. Show the relationship between log SCI and log distance in a figure. Highlight the points of at least one user geography (country or NUTS 2). Fit a curve to the data and plot it along with the scatterplot.

3. Visualize the distributions of log SCI by income bin (e.g. low, medium, high) of the user region. Comment.

Bonus: Explore the data and feel free to visualize whatever you would like. Make sure you understand the variables and compute summary statistics.

# Submission ⭐

Congratulations you finished Module 2! We will handle all exercise submissions on GitHub Classroom. Please push your changes to your team repository folder on GitHub Classroom.